In [1]:
%reload_kedro

2023-10-09 13:51:00,476 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\Dushku\PycharmProjects\monash-datahack-2023
2023-10-09 13:51:00,605 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-10-09 13:51:00,624 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-telemetry-0.2.5
2023-10-09 13:51:00,680 - kedro.extras.extensions.ipython - INFO - ** Kedro project Monash datahack 2023
2023-10-09 13:51:00,681 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\kedro\framework\context\context.py:344: UserWarning: Credentials not found in your Kedro project config.
No files found in ['C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\base', 'C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [2]:
catalog.list()

['raw_data',
 'preprocessed_data_with_type_classification',
 'preprocessed_data_with_total_classification',
 'preprocessed_data_not_recycled',
 'preprocessed_data_08_12_13',
 'preprocessed_data_not_08_12_13',
 'preprocessed_data_with_mining',
 'preprocessed_data_without_mining',
 'preprocessed_data_19_21',
 'preprocessed_data_not_19_21',
 'preprocessed_data_without_australia',
 'encoded_data_without_australia',
 'preprocessed_data_with_australia',
 'encoded_data_with_australia',
 'encoded_data_19_21',
 'encoded_data_not_19_21',
 'encoded_data_with_type_classification',
 'encoded_data_with_total_classification',
 'encoded_data_without_zero_values',
 'encoded_data_not_recycled',
 'encoded_data_08_12_13',
 'encoded_data_not_08_12_13',
 'encoded_data_with_mining',
 'encoded_data_without_mining',
 'companies',
 'reviews',
 'shuttles',
 'data_processing.preprocessed_companies',
 'data_processing.preprocessed_shuttles',
 'model_input_table',
 'data_science.active_modelling_pipeline.regressor'

In [3]:
df = catalog.load('preprocessed_data_with_type_classification')

2023-10-09 13:51:00,696 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_type_classification` (CSVDataSet)...


In [4]:
df.head()

,Jurisdiction,Category,Type,Management,Fate,y,ds,Sub-stream
0,ACT,Biosolids,Biosolids,Other disposal,Disposal,20186,2021-12-31,3
1,ACT,Biosolids,Biosolids,Recycling,Recycling,0,2021-12-31,3
2,ACT,Biosolids,Biosolids,Other disposal,Disposal,20186,2021-12-31,9
3,ACT,Biosolids,Biosolids,Recycling,Recycling,0,2021-12-31,9
4,ACT,Building and demolition materials,Asphalt,Recycling,Recycling,0,2021-12-31,2


In [5]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub-stream'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54227 entries, 0 to 54226
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Jurisdiction  54227 non-null  object
 1   Category      54227 non-null  object
 2   Type          54227 non-null  object
 3   Management    54227 non-null  object
 4   Fate          54227 non-null  object
 5   y             54227 non-null  int64 
 6   ds            54227 non-null  object
 7   Sub-stream    54227 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [7]:
df['y'] = df['y'].astype(int)

In [8]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [9]:
def encode(df):
    df['y'] = df['y'].astype(int)
    label_encoders = {}

    for col in df.columns:
        if df[col].dtype == 'object':  # Check if the column contains categorical data
            label_encoders[col] = LabelEncoder()
            df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])

    df['y'] = LabelEncoder().fit_transform(df['y'])
    
    return df

In [10]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub-stream', 'Jurisdiction_encoded', 'Category_encoded',
       'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [11]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54227 entries, 0 to 54226
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   y                     54227 non-null  int64
 1   Jurisdiction_encoded  54227 non-null  int32
 2   Sub-stream            54227 non-null  int64
 3   Category_encoded      54227 non-null  int32
 4   Type_encoded          54227 non-null  int32
 5   Management_encoded    54227 non-null  int32
 6   Fate_encoded          54227 non-null  int32
 7   ds_encoded            54227 non-null  int32
dtypes: int32(6), int64(2)
memory usage: 2.1 MB


In [13]:
catalog.save("encoded_data_with_type_classification", df)

2023-10-09 13:51:01,146 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_type_classification` (CSVDataSet)...


In [14]:
df = catalog.load('preprocessed_data_with_total_classification')

2023-10-09 13:51:01,199 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_total_classification` (CSVDataSet)...


In [15]:
df['y'] = df['y'].astype(int)

In [16]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [17]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Total type', 'Management', 'Fate',
       'y', 'ds', 'Sub-stream', 'Jurisdiction_encoded', 'Category_encoded',
       'Type_encoded', 'Total type_encoded', 'Management_encoded',
       'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [18]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Total type', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [19]:
catalog.save("encoded_data_with_total_classification", df)

2023-10-09 13:51:01,246 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_total_classification` (CSVDataSet)...


In [20]:
df = catalog.load('preprocessed_data_not_recycled')

2023-10-09 13:51:01,258 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_not_recycled` (CSVDataSet)...


In [21]:
df['y'] = df['y'].astype(int)

In [22]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [23]:
df.columns

Index(['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded',
       'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [24]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [25]:
df.head()

,y,Jurisdiction_encoded,Sub-stream,Category_encoded,Type_encoded,Management_encoded,Fate_encoded,ds_encoded
0,4748,0,3,1,7,2,0,14
1,4748,0,9,1,7,2,0,14
2,4748,0,3,1,68,2,0,14
3,4748,0,9,1,68,2,0,14
4,4748,0,3,1,68,2,0,14


In [26]:
catalog.save("preprocessed_data_not_recycled", df)

2023-10-09 13:51:01,325 - kedro.io.data_catalog - INFO - Saving data to `preprocessed_data_not_recycled` (CSVDataSet)...


In [27]:
preprocessed_data_names = ['preprocessed_data_08_12_13', 
                           'preprocessed_data_not_08_12_13', 
                           'preprocessed_data_with_mining', 
                           'preprocessed_data_without_mining',
                           'preprocessed_data_19_21',
                           'preprocessed_data_not_19_21',
                           'preprocessed_data_without_australia',
                          'preprocessed_data_with_australia']
encoded_data_names = ['encoded_data_08_12_13', 
                      'encoded_data_not_08_12_13', 
                      'encoded_data_with_mining', 
                      'encoded_data_without_mining',
                      'encoded_data_19_21',
                      'encoded_data_not_19_21',
                      'encoded_data_without_australia',
                      'encoded_data_with_australia',
                     ]

In [28]:
for i in range(len(preprocessed_data_names)):
    df = encode(catalog.load(preprocessed_data_names[i]))
    df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]
    catalog.save(encoded_data_names[i],df)

2023-10-09 13:51:01,380 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_08_12_13` (CSVDataSet)...
2023-10-09 13:51:01,396 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_08_12_13` (CSVDataSet)...
2023-10-09 13:51:01,404 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_not_08_12_13` (CSVDataSet)...
2023-10-09 13:51:01,470 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_not_08_12_13` (CSVDataSet)...
2023-10-09 13:51:01,518 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_mining` (CSVDataSet)...
2023-10-09 13:51:01,523 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_mining` (CSVDataSet)...
2023-10-09 13:51:01,525 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_without_mining` (CSVDataSet)...
2023-10-09 13:51:01,589 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_without_mining` (CSVDataSet)...
2023-10-09 13:51:01,638 - kedro.io.data_catalo